**GROUP PROJECT PROPOSAL**


# **USING AGE, HEARTRATE, AND CHOLESTROL TO CLASSIFY HEART PATIENTS IN CLEVELAND**
#### ARYAN JAIN, KATIE LIANG, MADILYN PORTAS, VIBHAV

## Introduction

**Heart disease** is a leading cause of death globally. Early detection can lead to effective treatment and management, potentially reducing the mortality rate. Our project aims to answer the question: “Can we predict the presence of heart disease based on health parameters?”

We will use the Heart Disease dataset from the UCI Machine Learning Repository. This dataset contains 76 attributes, including age, sex, chest pain type, resting blood pressure, cholesterol levels, fasting blood sugar, and others.

In [4]:
#Importing Libraries
library(tidyverse)
library(tidymodels)
library(repr)
library(RColorBrewer)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ broom        1.0.2     ✔ rsample      1.1.1
✔ dials        1.1.0     ✔ tune         1.0.1
✔ infer        1.0.4     ✔ workflows    1.1.2
✔ modeldata    1.0.1     ✔ workflowsets 1.0.0
✔ parsnip      1.0.3     ✔ yardstick    1.1.0
✔ recipes      1.0.4     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Learn how to get started at https://www.tidymodels.org/start/

